In [1]:
import pathlib
import pandas as pd

In [2]:
data_path = pathlib.Path().absolute() / "data"
pbp_data = data_path.glob("play_by_play_*.csv")
pbp_df = pd.concat(map(pd.read_csv, pbp_data))
pbp_df.shape

(1148717, 372)

In [3]:
mask = pbp_df["play_type"].isin(["pass", "run"])
pbp_df = pbp_df.loc[mask]
pbp_df.shape

(803966, 372)

In [4]:
def coach_from_posteam(row):
    if row['posteam'] == row['home_team']:
        return row['home_coach']
    elif row['posteam'] == row['away_team']:
        return row['away_coach']
    raise ValueError("'posteam' not equal to 'home_team' or 'away_team'.")

pbp_df['coach'] = pbp_df.apply(coach_from_posteam, axis=1)
pbp_df['coach'].value_counts()

coach
Bill Belichick      26796
Andy Reid           26420
Mike McCarthy       17572
Mike Tomlin         17392
Jeff Fisher         17375
                    ...  
Perry Fewell          383
Mel Tucker            302
Eric Studesville      250
Emmitt Thomas         217
Jay Rosburg           128
Name: count, Length: 158, dtype: int64

In [6]:
pbp_df["play_in_drive"] = pbp_df.groupby(["game_id", "drive"]).cumcount() + 1
pbp_df.shape

(803966, 374)

In [8]:
columns = [
    'play_type',
    'posteam',
    'coach',
    'drive',
    'play_in_drive',
    'qtr',
    'quarter_seconds_remaining',
    'down',
    'ydstogo',
    'yardline_100',
    'shotgun',
    'no_huddle',
    'score_differential',
    'spread_line',
    'season',
]

final_df = pbp_df.loc[:, columns]
final_df = final_df[final_df.qtr <= 5]
final_df = final_df.dropna()

final_df.head()

,play_type,posteam,coach,drive,play_in_drive,qtr,quarter_seconds_remaining,down,ydstogo,yardline_100,shotgun,no_huddle,score_differential,spread_line,season
1,pass,PHI,Andy Reid,1.0,1.0,1,900.0,1.0,10,77.0,0,0,0.0,-3.0,1999
2,run,PHI,Andy Reid,1.0,2.0,1,900.0,2.0,10,77.0,0,0,0.0,-3.0,1999
4,pass,PHI,Andy Reid,1.0,3.0,1,900.0,3.0,14,81.0,0,0,0.0,-3.0,1999
6,run,ARI,Vince Tobin,2.0,1.0,1,787.0,1.0,10,81.0,0,0,0.0,-3.0,1999
7,run,ARI,Vince Tobin,2.0,2.0,1,787.0,2.0,9,80.0,0,0,0.0,-3.0,1999


In [9]:
final_df.to_csv('data/play_by_play_parsed.csv', index=False)